# Final Agent
Okay, now we're cooking. Our agent can access some data, that's super cool! But it's not the best now, is it? It can access data, but it loads the ENTIRE dataset into the chat! That's not practical. Especially if our dataset as very large.

What we'll do next is to QUERY data SMARTLY! We'll have agents write the SQL to query our data for us. And that will be the final form of our agent.

---

In this module, we'll make up some large mock dataset, and have the agent query from that as necessary with the SQL query that seems appropriate.

## 1 Configs

### 1.1 Installs

* `dotenv` – loads and manages environment variables from a `.env` file.

* `duckdb` – runs fast SQL queries directly on Pandas DataFrames or local files.

* `langchain-core` – the lightweight core of LangChain, providing the `Runnable` interfaces and core abstractions.

* `langchain_mcp_adapters` – bridges LangChain with MCP servers, letting agents call remote tools securely. (Only needed if dealing with MCP Servers)

* `langgraph` – builds stateful agent workflows as directed graphs of nodes and edges.

* `pandas` – flexible, high-performance data analysis and manipulation library.

* `requests` – simple HTTP client for making API calls (e.g., to Databricks endpoints).

* `truststore` – ensures Python’s SSL connections use your system’s trusted certificate store.

In [ ]:
%%capture
%pip install \
   dotenv \
   duckdb \
  "langchain-core==0.3.79" \
  "langchain_mcp_adapters==0.1.11" \
  "langgraph==0.2.41" \
  pandas \
  requests \
  truststore 


In [ ]:
# restart kernel to use newly installed packages
%reset -f

### 1.2 Imports

 - from `langgraph.graph import StateGraph, END`
   - `StateGraph` - build and compile our node graph.
   - `END` - sentinel that tells LangGraph where to stop.

 - `from langchain_core.runnables import RunnableLambda` - wraps a normal Python function so the graph can call it like any other LangChain “runnable.”

 - `from typing import Dict, List, Optional, TypedDict` - creates AgentState, a typed dictionary that documents (and type-checks) the keys we pass between nodes.

 - `my_functions.databricks_llm` - Our custom useful function to make llm requests

 - `os` - Access environment variables that store our Azure configuratio

 - `dotenv.load_dotenv` - Reads the `.env` file and safely loads the environmental variables

 - `json` - Package to manage json-style strings/objects

In [ ]:
# LangGraph core components for building agent workflows
from langgraph.graph import StateGraph, END       # StateGraph: main graph builder, END: termination signal
from langchain_core.runnables import RunnableLambda  # Converts functions to graph-compatible nodes

# Type annotations for better code clarity and IDE support
from typing import Dict, List, Optional, TypedDict

# Utility and system packages
from dotenv import load_dotenv
import os
import json

# Our custom utility functions from the previous notebook
from my_functions import databricks_llm

# Dataframe handling functions
import pandas as pd
import duckdb as ddb

### 1.3 Loading Environmental Variables

Load our Azure OpenAI configuration from the secure `.env` file.

In [ ]:
# Load environment variables (Azure endpoints, API versions, etc.)
load_dotenv(".env", override=True)

## 2 Defining Functions and Classes

### 2.1 Classes

Here, let's change the `AgentState` class just a little mode:
 - We'll just add the field `schema_catalog`, so any node of the graph can consult it if needed. This should hold the schema of our database.

In [ ]:
class AgentState(TypedDict, total=False):
    """
    Enhanced state container for our tool-enabled agent.
    
    Carries conversation data and tool information between graph nodes.
    """
    # Core fields for tools and conversation (from previous notebook)
    chat_history: List[Dict[str, str]]        # Complete conversation in OpenAI format
    output: Optional[str]                     # Most recent response
    available_tools: Optional[Dict[str, str]] # Tool names → descriptions for router
    tool_context: Optional[str]               # Context retrieved by tools (cleared each turn)

    # New fields for database communication
    schema_catalog: Optional[Dict[str, str]] # catalog of schemas and their descriptions

### 2.2 Agent Functions with Tool Support

**Three Specialized Agent Functions:**

**1. Router Agent (`router_agent`)**
- **Purpose**: Decides which tool to use based on user's query
- **Input**: Chat history + available tools
- **Output**: JSON decision like `{"tool": "email"}` or `{"tool": "chat"}`
- **Key Feature**: Uses its own system prompt to focus on tool selection

**2. Sales Data Agent (`sales_data_agent`)**  
- **Purpose**: Looks at chat history and writes SQL query to answer user's question
- **Process**: Adds results to `tool_context`

**3. Sales Data Tool (`sales_data_tool`)**
- **Purpose**: Runs the SQL query created by the sales_data_agent
- **Process**: Uses duckdb to run the SQL query on the databased loaded by pandas
- **No LLM needed**: Pure data retrieval/querying

**4. Chat Agent (`chat_agent`)**
- **Purpose**: Generates final response using conversation + tool context

#### 2.2.1 - Router Agent

In [ ]:
def router_agent(state):
    """
    Router agent that decides which tool to use based on the user's query.
    
    This agent analyzes the conversation and available tools to make an 
    intelligent routing decision.
    """
    print("\n--- ROUTER AGENT NODE ---")
    print(f"Analyzing user query with {len(state.get('available_tools', {}))} available tools")

    # === BUILD TOOL CATALOG FOR LLM ===
    # Create a formatted list of available tools and their descriptions
    tool_lines = [
        f"- {name}: {desc}"
        for name, desc in (state["available_tools"] or {}).items()
    ]
    tool_catalog = "\n".join(tool_lines) or "none"

    # === CREATE ROUTER-SPECIFIC SYSTEM PROMPT ===
    # The router has a specialized role: tool selection only
    router_system_prompt = (
        "You are an AI router. Choose the single best tool for answering the user's "
        "latest message.\n\n"
        f"Available tools:\n{tool_catalog}\n\n"
        "Return ONLY a JSON object like {\"tool\": \"chat\"} or {\"tool\": \"sales_data\"}."
    )

    # === PREPARE MODIFIED CHAT HISTORY ===
    # Replace system prompts with router-specific prompt
    # This ensures the LLM focuses on tool selection, not general chat
    modified_chat_history = [
        {"role": "system", "content": router_system_prompt}
    ] + [
        m for m in state["chat_history"] 
        if m["role"] != "system"  # Filter out original system prompts
    ]

    print(f"Sending {len(modified_chat_history)} messages to LLM for routing decision")

    # === GET ROUTING DECISION FROM LLM ===
    llm_response = databricks_llm(modified_chat_history, os.getenv("INSTRUCT_ENDPOINT"))
    
    print(f"LLM routing response: {llm_response}")

    # === EXTRACT JSON DECISION ===
    # Parse JSON from LLM response (handle any extra text)
    start = llm_response.rfind("{")
    end   = llm_response.rfind("}")
    
    if start == -1 or end == -1:
        # Fallback to chat if no valid JSON found
        decision = {"tool": "chat"}
        print("⚠️  No valid JSON found, defaulting to chat")
    else:
        decision_json = llm_response[start : end + 1]
        try:
            decision = json.loads(decision_json)
            print(f"✓ Extracted decision: {decision}")
        except json.JSONDecodeError:
            decision = {"tool": "chat"}
            print("⚠️  JSON parse error, defaulting to chat")

    # === UPDATE STATE ===
    # Store decision as JSON string for conditional edges
    state["output"] = json.dumps(decision)

    print("--- ROUTER AGENT NODE END ---\n")
    return state

#### 2.2.2 - Sales Data Agent

In [ ]:
def sales_data_agent(state):
    """
    Sales Data agent that writes SQL to answer user questions against the sales schema.

    This agent:
      1) Injects a schema-aware system prompt,
      2) Strips other system prompts from history,
      3) Calls the LLM to produce a single SQL query,
      4) Robustly extracts a fenced SQL block,
      5) Applies light guardrails (ensure SELECT, remove trailing semicolons),
      6) Stores the final SQL in state["output"].
    """
    print("\n--- SALES DATA AGENT NODE ---")

    # === LOAD SCHEMA ===
    # Expecting a schema catalog shaped like: {"sales_data": "...DDL and table/column docs..."}
    schema_catalog = state.get("schema_catalog", {}) or {}
    schema = schema_catalog.get("sales_data")
    if not schema:
        print("⚠️  No sales_data schema found; falling back to minimal instruction.")
        schema = "/* Schema unavailable: write a best-effort ANSI SQL query. */"

    # === BUILD SALES-SPECIFIC SYSTEM PROMPT ===
    sales_system_prompt = (
        "You are a focused SQL authoring assistant for SALES data.\n"
        "Goal: write ONE SQL query that answers the user's latest request using the schema below.\n\n"
        f"Schema:\n{schema}\n\n"
        "Constraints:\n"
        "- Output ONLY a single SQL query, no prose.\n"
        "- Wrap it exactly like:\n"
        "```sql\nSELECT ...\n```\n"
    )

    # === PREPARE MODIFIED CHAT HISTORY ===
    # Replace any prior system prompts with our sales-specific instructions
    modified_chat_history = (
        [{"role": "system", "content": sales_system_prompt}] +
        [m for m in state.get("chat_history", []) if m.get("role") != "system"]
    )
    print(f"Sending {len(modified_chat_history)} messages to LLM")

    # === CALL LLM ===
    llm_response = databricks_llm(
        modified_chat_history,
        model_endpoint=os.getenv("CHAT_ENDPOINT")
    )
    print("Raw LLM response received.")

    # === EXTRACT FENCED SQL BLOCK ===
    # Look for the last fenced block to be resilient to accidental extra text.
    start = llm_response.rfind("```")
    end = len(llm_response)
    # If we found a closing fence earlier, prefer the last full triple-backtick pair:
    if start != -1:
        # Find the opening of that block
        open_start = llm_response.rfind("```", 0, start)
        if open_start != -1:
            code_block = llm_response[open_start + 3:start].strip()
        else:
            # Only one fence found—fall back to substring after fence
            code_block = llm_response[start + 3:].strip()
    else:
        # No fences—fallback: try to salvage any SELECT ...; looking substring
        print("⚠️  No fenced code block found; attempting fallback extraction.")
        code_block = llm_response.strip()

    # Remove optional "sql" language tag at the top of the fenced block
    if code_block.lower().startswith("sql"):
        code_block = code_block[3:].strip()

    # === LIGHT GUARDRAILS ===
    sql = code_block.strip()
    # If the block still contains fencing, strip again
    if sql.startswith("```") and sql.endswith("```"):
        sql = sql[3:-3].strip()

    # Require a SELECT to reduce the chance of non-query outputs
    if "select" not in sql.lower():
        print("⚠️  No SELECT found; defaulting to a safe placeholder query.")
        sql = "SELECT 1 /* fallback: LLM did not produce a valid SELECT */"

    # Normalize: remove trailing semicolon for consistency in some executors
    sql = sql.rstrip().rstrip(";").strip()

    print("Generated SQL:\n", sql)

    # === UPDATE STATE ===
    state["output"] = sql

    print("--- SALES DATA AGENT NODE END ---\n")
    return state


#### 2.2.3 - Sales Data Tool

In [ ]:
def sales_data_tool(state):
    """
    Sales Data Tool Node

    This tool executes the SQL query generated by the Sales Data Agent
    against a local in-memory dataset using DuckDB. It converts the result
    into a readable text context for downstream LLM processing.
    """
    print("\n--- SALES DATA TOOL NODE ---")

    # === EXTRACT SQL QUERY FROM STATE ===
    # The previous agent node is expected to store its output SQL here
    query = state.get("output", "").strip()
    if not query:
        print("⚠️  No SQL query found in state; skipping execution.")
        state["tool_context"] = "/* No SQL query to execute */"
        return state

    # === LOAD DATA ===
    # Load the mock dataset from CSV into a DataFrame
    try:
        sales_data = pd.read_csv("data/mock_sales_data.csv")
        print(f"Loaded {len(sales_data):,} rows from mock_sales_data.csv")
    except Exception as e:
        print(f"❌ Failed to load data: {e}")
        state["tool_context"] = f"/* Data load error: {e} */"
        return state

    # === EXECUTE SQL USING DUCKDB ===
    print("Executing SQL with DuckDB:\n", query)
    try:
        # Run query against in-memory DataFrame
        result_df = ddb.query_df(sales_data, "sales_data", query).to_df()
        print(f"✓ Query executed successfully, returned {len(result_df):,} rows.")
    except Exception as e:
        print(f"❌ Query execution failed: {e}")
        result_df = pd.DataFrame({"error": [str(e)]})

    # === PREPARE CONTEXT STRING FOR LLM ===
    # Convert the DataFrame to a human-readable string (no index)
    context = "QUERY: " + query + "\nANSWER: " + result_df.to_string(index=False)
    state["tool_context"] = context
    state["output"] = context

    print("SQL Result:\n", context)

    print("--- SALES DATA TOOL NODE END ---\n")

    # Return updated state for next node
    return state


#### 2.2.4 - Chat Agent

In [ ]:
def chat_agent(state):
    """
    Enhanced chat agent that incorporates tool context into responses.
    
    This agent generates the final response using both the conversation
    history and any additional context provided by tools.
    """
    print("\n--- CHAT AGENT NODE ---")
    
    # === PREPARE ENHANCED CHAT HISTORY ===
    # Start with the original conversation
    enhanced_chat_history = state["chat_history"].copy()
    
    # === ADD TOOL CONTEXT IF AVAILABLE ===
    if state.get("tool_context"):
        print("✓ Including tool context in conversation")
        # Add tool context as a user message so the AI can reference it
        context_message = {
            "role": "user", 
            "content": f"ADDITIONAL CONTEXT FROM TOOLS:\n{state['tool_context']}"
        }
        enhanced_chat_history.append(context_message)
    else:
        print("No tool context available")

    print(f"Sending {len(enhanced_chat_history)} messages to LLM")
    
    # === GENERATE RESPONSE ===
    reply = databricks_llm(enhanced_chat_history, os.getenv("CHAT_ENDPOINT"))
    
    # === UPDATE STATE ===
    # Add only the actual AI response to the permanent chat history
    # (Don't include the temporary tool context message)
    state["chat_history"].append({"role": "assistant", "content": reply})
    state["output"] = reply

    print(f"✓ Generated response: {reply[:100]}...")
    print("--- CHAT AGENT NODE END ---\n")

    return state

# 3 - Initializing Chat

Now we'll create the graph for the agent

### 3.1 Graph with Conditional Routing, Agents, and Tool

**Our Graph Flow:**
```
START → router_agent → [decision] → sales_data_agent → sales_data_tool → chat_agent → END
                           ↓
                       chat_agent → END
```

In [ ]:
# === STEP 1: INITIALIZE GRAPH ===
g = StateGraph(AgentState)
print("✓ Graph initialized with enhanced AgentState")

# === STEP 2: ADD NODES ===
# Add all nodes: router, sales SQL author, SQL executor, and final chat responder
g.add_node("router_agent",      RunnableLambda(router_agent))       # Decision maker
g.add_node("sales_data_agent",  RunnableLambda(sales_data_agent))   # SQL author
g.add_node("sales_data_tool",   RunnableLambda(sales_data_tool))    # SQL executor
g.add_node("chat_agent",        RunnableLambda(chat_agent))         # Response generator
print("✓ Added 4 nodes: router_agent, sales_data_agent, sales_data_tool, chat_agent")

# === STEP 3: SET ENTRY POINT ===
# Always start with the router to choose the best path
g.set_entry_point("router_agent")
print("✓ Set router_agent as entry point")

# === STEP 4: DEFINE ROUTE SELECTOR FUNCTION ===
def route_selector(state: AgentState) -> str:
    """
    Parse the router's decision and return the next node name.

    Args:
        state: Current agent state containing router's JSON decision in state["output"]

    Returns:
        str: Node key ("chat" or "sales_data"); defaults to "chat" on errors.
    """
    try:
        decision = json.loads(state["output"]) if state.get("output") else {}
        tool_choice = decision.get("tool", "chat")
    except Exception as e:
        print(f"⚠️  Route selector failed to parse decision ({e}); defaulting to 'chat'")
        tool_choice = "chat"

    print(f"Route selector: directing to '{tool_choice}'")
    return tool_choice

# === STEP 5: ADD CONDITIONAL EDGES ===
# Router directs either to chat or the sales data SQL authoring agent
g.add_conditional_edges(
    "router_agent",        # Source node
    route_selector,        # Function that decides the route
    {
        "chat":       "chat_agent",        # Simple questions → chat
        "sales_data": "sales_data_agent",  # Data questions → sales SQL author
    },
)
print("✓ Added conditional edges from router_agent")

# === STEP 6: ADD SIMPLE EDGES ===
# Sales author → Sales tool → Chat → END
g.add_edge("sales_data_agent", "sales_data_tool")
print("✓ Added edge: sales_data_agent → sales_data_tool")

g.add_edge("sales_data_tool", "chat_agent")
print("✓ Added edge: sales_data_tool → chat_agent")

g.add_edge("chat_agent", END)
print("✓ Added edge: chat_agent → END")

# === STEP 7: COMPILE THE GRAPH ===
assistant_graph = g.compile()
print("✓ Graph compiled successfully!")

print("\nGraph structure:")
print("START → router_agent → [decision]")
print("                         ├─ 'chat'       → chat_agent → END")
print("                         └─ 'sales_data' → sales_data_agent → sales_data_tool → chat_agent → END")
print("\nThe tool-enabled agent is ready!")

### 3.2 Interactive Smart Querying Chat

In [ ]:
# === INITIALIZE ENHANCED CONVERSATION STATE ===
chat_history = [
    {"role": "system", "content": "You are a helpful AI Agent. You have access to a sales data database if needed."}
]

schema_catalog = {
    "sales_data": (
        "The 'sales_data' table has the following columns:\n"
        "- client (string): the customer name\n"
        "- product (string): the item sold\n"
        "- quantity (int): number of units sold\n"
        "- price (float): price per unit\n"
        "- date (datetime): date of transaction\n"
    )
}

state = AgentState(
    chat_history=chat_history,
    output=None,
    # Define available tools for the router to choose from
    available_tools={
        "chat": "Continue the conversation naturally",
        "sales_data": "Query the internal sales database using SQL"
    },
    tool_context=None,  # Will be populated by tools when needed
    schema_catalog=schema_catalog
)

print("🤖 Tool-enabled AI Agent initialized!")
print("Available tools: sales_data search, regular chat")
print("Type 'exit' to quit the conversation.\n")

# === MAIN ENHANCED CHAT LOOP ===
while True:
    # === GET USER INPUT ===
    user_text = input("You: ").strip()
    
    # === CHECK FOR EXIT ===
    if user_text.lower() == "exit":
        break
    
    if not user_text:
        continue
    
    print(f"\n📝 Processing: '{user_text}'")
    
    # === ADD USER MESSAGE TO CONVERSATION ===
    state["chat_history"].append({"role": "user", "content": user_text})
    
    # === EXECUTE THE ENHANCED GRAPH ===
    # This will:
    # 1. Route through router_agent to decide on tools
    # 2. Optionally use sales_data_agent + sales_data_tool to get context
    # 3. Generate final response with chat_agent
    print("🔄 Running enhanced agent graph...")
    state = assistant_graph.invoke(state)
    
    # === CLEANUP TOOL CONTEXT ===
    # Clear tool context after each interaction to prevent carryover
    state["tool_context"] = None
    
    # === DISPLAY RESPONSE ===
    print(f"🤖 Assistant: {state['output']}")
    print("-" * 60)

## 4 - A Nod to MCP

In the same spirit as the previous notebook, I'll add an example using MCP.

This is an even better application, where the heavy-duty SQL queries can happen on remote databases that are much larger and might required better methods than `duckdb`.

### 4.1 Imports

We'll put it all nice and neat in the `my_mcp_functions.py`

In [ ]:
from my_mcp_functions import list_tools, call_tool

### 4.2 Incorporating MCP Into our Agent

In [ ]:
# Note that it's "async" now, as it has API calls
async def mcp_sales_data_tool(state):
    print("\n--- MCP SALES DATA TOOL NODE ---")

    query = state.get("output", "").strip()
    if not query:
        print("⚠️  No SQL query found in state; skipping execution.")
        state["tool_context"] = None
        return state

    result = await call_tool("query_sales_data", {'sql_query': query})

    context = "QUERY: " + query + "\nANSWER: " + result

    state["tool_context"] = context
    state["output"] = context

    print("SQL Result:\n", context)

    print("--- MCP SALES DATA TOOL NODE END ---\n")

    return state


### 4.3 Redefining Graph with New Tool

In [ ]:
g = StateGraph(AgentState)
g.add_node("router_agent",      RunnableLambda(router_agent))       
g.add_node("sales_data_agent",  RunnableLambda(sales_data_agent))
g.add_node("sales_data_tool",   RunnableLambda(mcp_sales_data_tool)) # <--- New Tool in
g.add_node("chat_agent",        RunnableLambda(chat_agent))

g.set_entry_point("router_agent")

def route_selector(state: AgentState) -> str:
    try:
        decision = json.loads(state["output"]) if state.get("output") else {}
        tool_choice = decision.get("tool", "chat")
    except Exception as e:
        tool_choice = "chat"
    return tool_choice

g.add_conditional_edges(
    "router_agent",
    route_selector,
    {
        "chat":       "chat_agent",
        "sales_data": "sales_data_agent",
    },
)
g.add_edge("sales_data_agent", "sales_data_tool")
g.add_edge("sales_data_tool", "chat_agent")
g.add_edge("chat_agent", END)

assistant_graph = g.compile()

### 4.4 Chat Loop with MCP Tools

In [ ]:
chat_history = [
    {"role": "system", "content": "You are a helpful AI Agent. You have access to a sales data database if needed."}
]

schema_catalog = {
    "sales_data": (
        "The 'sales_data' table has the following columns:\n"
        "- client (string): the customer name\n"
        "- product (string): the item sold\n"
        "- quantity (int): number of units sold\n"
        "- price (float): price per unit\n"
        "- date (datetime): date of transaction\n"
    )
}

state = AgentState(
    chat_history=chat_history,
    output=None,
    available_tools={
        "chat": "Continue the conversation naturally",
        "sales_data": "Query the internal sales database using SQL"
    },
    tool_context=None,
    schema_catalog=schema_catalog
)

print("🤖 Tool-enabled AI Agent initialized!")
print("Available tools: sales_data search, regular chat")
print("Type 'exit' to quit the conversation.\n")

while True:
    user_text = input("You: ").strip()
    
    if user_text.lower() == "exit":
        break
    
    if not user_text:
        continue
    
    print(f"\n📝 Processing: '{user_text}'")
    state["chat_history"].append({"role": "user", "content": user_text})
    print("🔄 Running enhanced agent graph...")
    state = await assistant_graph.ainvoke(state) # <--- Only change for asynchronous execution
    
    state["tool_context"] = None
    
    print(f"🤖 Assistant: {state['output']}")
    print("-" * 60)